<a href="https://colab.research.google.com/github/n-vit/YaP_Projects/blob/main/6_%D0%92%D0%B2%D0%B5%D0%B4%D0%B5%D0%BD%D0%B8%D0%B5_%D0%B2_%D0%BC%D0%B0%D1%88%D0%B8%D0%BD%D0%BD%D0%BE%D0%B5_%D0%BE%D0%B1%D1%83%D1%87%D0%B5%D0%BD%D0%B8%D0%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Рекомендация тарифов

Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится.

Нужно построить модель с максимально большим значением *accuracy* - довести долю правильных ответов по крайней мере до 0.75. 

In [ ]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression 
from sklearn.metrics import accuracy_score 
from sklearn.model_selection import train_test_split
import time


## Откройте и изучите файл

In [ ]:
df = pd.read_csv('/datasets/users_behavior.csv')
display(df.head(3))

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0


In [ ]:
display(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


None

<div class="alert alert-info", style="color:brown">

теперь видно, что во фрейме 3214 строк, предобработка проведена, можно бы понизить разрядность, возможно модельки будут быстрее работать
    
</div>

In [ ]:
df_float = df.select_dtypes(include=['float']) 
df_float = df_float.apply(pd.to_numeric,downcast='float')
df[df_float.columns] = df_float

df_int = df.select_dtypes(include=['int']) 
df_int = df_int.apply(pd.to_numeric,downcast='unsigned') 
df[df_int.columns] = df_int

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float32
 1   minutes   3214 non-null   float32
 2   messages  3214 non-null   float32
 3   mb_used   3214 non-null   float32
 4   is_ultra  3214 non-null   uint8  
dtypes: float32(4), uint8(1)
memory usage: 53.5 KB


<div class="alert alert-info", style="color:brown">

фрейм стал в два раза полегче
    
</div>

## Разбейте данные на выборки

Разделим данные на три выборки:
- тренировочную (чтобы обучать модели) - 60%, 
- валидационную (чтобы оценивать результаты обучения) - 20%, 
- тестовую (чтобы сравнить модели между собой) - 20%

In [ ]:
features = df.drop(['is_ultra'], axis=1)
target = df['is_ultra']

features_train, features_a, target_train, target_a = train_test_split(
    features, target, test_size=0.4, random_state=12345)

features_valid, features_test, target_valid, target_test = train_test_split(
    features_a, target_a, test_size=0.5, random_state=12345)


In [ ]:
print('Фичи обучающей выборки', features_train.shape)
print('Таргет обучающей выборки', target_train.shape)
print('Фичи валидационной выборки', features_valid.shape)
print('Таргет валидационной выборки', target_valid.shape)
print('Фичи тестовой выборки', features_test.shape)
print('Таргет тестовой выборки', target_test.shape)

Фичи обучающей выборки (1928, 4)
Таргет обучающей выборки (1928,)
Фичи валидационной выборки (643, 4)
Таргет валидационной выборки (643,)
Фичи тестовой выборки (643, 4)
Таргет тестовой выборки (643,)


<div class="alert alert-info", style="color:brown">

Визуально вроде 60/20/20, едем дальше
    
</div>

## Исследуйте модели

Поскольку наша цель - сгруппировать пользователей по одному из двух тарифов, а не определить количественные данные - будем использовать **модели классификации**: Решающее дерево, Случайный лес и Логистическую регрессию. Качество модели будем оценивать с помощью `accuracy`

начнем с **Решающего дерева**, и определим для него глубину ветвления max_depth:

In [ ]:
model_dtc = None
best_result = 0
best_depth = 0
for depth in range(1, 10):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model.fit(features_train, target_train)
    result = model.score(features_valid, target_valid)
    if result > best_result:
        model_dtc = model
        best_result = result
        best_depth = depth
print("Accuracy наилучшей модели `DecisionTree` на валидационной выборке:", best_result)
print('Оптимальный параметр max_depth:', best_depth)


Accuracy наилучшей модели `DecisionTree` на валидационной выборке: 0.7853810264385692
Оптимальный параметр max_depth: 3


И сразу хороший результат, хотя-бы одна модель соответствующая условию задачи (Accuracy > 0,75) у нас уже есть

проверим модель **Случайного леса**. Поскольку у нас исследовательская задача и временем мы располагаем - можно поэкспериментировать с количеством оценщиков и выбрать лучшую модель. 

In [ ]:
model_rfc = None
best_result = 0
best_est = 0
for est in range(1, 50):
    model = RandomForestClassifier(random_state=12345, n_estimators=est)
    model.fit(features_train, target_train) 
    result = model.score(features_valid, target_valid)
    if result > best_result:
        model_rfc = model
        best_result = result
        best_est = est

print("Accuracy наилучшей модели на валидационной выборке:", best_result)
print('Оптимальный параметр n_estimators:', best_est)


Accuracy наилучшей модели на валидационной выборке: 0.7947122861586314
Оптимальный параметр n_estimators: 23


Значение `accuracy` выше 0.79. Результат так же удовлетворяет поставленной задаче

Посмотрим на работу модели **Логистической регрессии:**

In [ ]:
model_lr = LogisticRegression(random_state=12345) 
model_lr.fit(features_train, target_train)
train_predictions = model_lr.predict(features_train)
valid_predictions = model_lr.predict(features_valid)
print('Accuracy модели `LogisticRegression` на валидационной выборке', accuracy_score(target_valid, valid_predictions))

Accuracy модели `LogisticRegression` на валидационной выборке 0.7107309486780715


А у этой модели  Accuracy ниже необходимого. Посмотрим что будет в сравнении моделей на тестовой выборке

<div class="alert alert-info", style="color:brown">

**По результатам валидации лучший Accuracy показала модель Случайного леса**, но результат незначительно отличается от модели Решающего дерева, а модель скорее всего работает дольше. Значит проверку времени работы кода и выбор модели стоит перенести в этот раздел. Так и сделаем, используя валидационную выборку:
    
</div>

In [ ]:
start = time.time()
train_predictions = model_dtc.predict(features_train)
valid_predictions = model_dtc.predict(features_valid)
end = time.time()
print('Время работы модели DecisionTree',end - start)

start = time.time()
train_predictions = model_rfc.predict(features_train)
valid_predictions = model_rfc.predict(features_valid)
end = time.time()
print('Время работы модели RandomForest',end - start)

Время работы модели DecisionTree 0.003572702407836914
Время работы модели RandomForest 0.017900466918945312


<div class="alert alert-info", style="color:brown">

Интересно, что сокращение разрядности фрейма практически не изменило времени работы кода, а Случайный лес на валидационной выборке работал даже дольше (0,018 вместо 0,016)

**Итого - для исследовательских целей оставляем Случайный лес**, т.к. он дает большую точность, а времени у нас достаточно. 
Если в бизнес-задаче будут лимиты по времени код - стоит рассматривать Решающее дерево с глубиной ветвления 3 как разумную альтернативу
    
</div>

## Проверьте модель на тестовой выборке

In [ ]:
#train_predictions = model_lr.predict(features_train)
#test_predictions = model_lr.predict(features_test)
#print('Accuracy Логистической регрессии на тестовой выборке', accuracy_score(target_test, test_predictions))

train_predictions = model_rfc.predict(features_train)
test_predictions = model_rfc.predict(features_test)
print('Accuracy Случайного леса на тестовой выборке', accuracy_score(target_test, test_predictions))

#train_predictions = model_dtc.predict(features_train)
#test_predictions = model_dtc.predict(features_test)
#print('Accuracy Решающего дерева на тестовой выборке', accuracy_score(target_test, test_predictions))

Accuracy Случайного леса на тестовой выборке 0.7807153965785381


In [ ]:
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.8, random_state=12345)
model = RandomForestClassifier(random_state=12345, n_estimators=23)
model.fit(features_train, target_train) 
train_predictions = model_rfc.predict(features_train)
test_predictions = model_rfc.predict(features_test)
print('Accuracy Случайного леса на тестовой выборке', accuracy_score(target_test, test_predictions))

Accuracy Случайного леса на тестовой выборке 0.8895800933125972


<div class="alert alert-info", style="color:brown">

Бинго! 89% попаданий! хороший способ, спасибо)
    
</div>

В тестовой выборке лучшие результаты показала модель `RandomForest` - с параметром `n_estimators`= 23, но результаты `DecisionTree` отличаются совсем незначительно. Чтобы решить стоит ли городить лес, или достаточно одного дерева - сверим время работы кода в бонусном разделе

## (бонус) Проверьте модели на адекватность

In [ ]:
len(df.query('is_ultra == 0')) / len(df)

0.693528313627878

<div class="alert alert-info", style="color:brown">

Если примитивная модель будет всегда предсказывать "0", то она попадет в 69% случаев. Тоже неплохая точность, но наши модели на основе решающего дерева или случайного леса будут полезнее. И даже логистическая регрессия покажет результат лучше, но наверное это сложно будет заметить с точки зрения бизнеса а не математики. 
    
    
Мы предложили модель которая дает всего 11% неверных ответов, а это заметно лучше чем просто предложить всем недорогой тариф и получить 31% ошибок. 
    
</div>

Значения Accuracy в работе каждой из трех моделей выше 0,5 (Accuracy случайного выбора), а значит решения каждой модели адекватны.

Мы можем попробовать еще улучшить Accuracy модели случайного леса, увеличив диапазон выбора n_estimators:

In [ ]:
start = time.time()
train_predictions = model_dtc.predict(features_train)
test_predictions = model_dtc.predict(features_test)
end = time.time()
print('Время работы модели DecisionTree',end - start)

start = time.time()
train_predictions = model_rfc.predict(features_train)
test_predictions = model_rfc.predict(features_test)
end = time.time()
print('Время работы модели RandomForest',end - start)

Время работы модели DecisionTree 0.0039920806884765625
Время работы модели RandomForest 0.033158302307128906


~Случайный лес из 23 деревьев работает в четыре раза дольше, чем решающее дерево с глубиной ветвления 3. А разница в Accuracy составляет одну тысячную.~ 

~**ВЫВОД**~
~В качестве модели для оценки поведения клиентов на соответствие тарифному плану рекомендуется обученная на 60% выборки модель DecisionTree с парамтером max_depth = 3, с Accuracy = 0.7791601866251944~
<div class="alert alert-info", style="color:brown">

Этот вывод уже не считается, т.к. мы остановились в Случайном лесу и дообучили его до Accuracy = 0,89
    
</div>


## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x] Jupyter Notebook открыт
- [x] Весь код исполняется без ошибок
- [x] Ячейки с кодом расположены в порядке исполнения
- [x] Выполнено задание 1: данные загружены и изучены
- [x] Выполнено задание 2: данные разбиты на три выборки
- [x] Выполнено задание 3: проведено исследование моделей
    - [x] Рассмотрено больше одной модели
    - [x] Рассмотрено хотя бы 3 значения гипепараметров для какой-нибудь модели
    - [x] Написаны выводы по результатам исследования
- [x] Выполнено задание 3: Проведено тестирование
- [x] Удалось достичь accuracy не меньше 0.75


<img src="http://s3.amazonaws.com/pix.iemoji.com/images/emoji/apple/ios-12/256/waving-hand.png" align=left width=44, heigth=44>
<div class="alert alert-info">
<b> Заключительный комментарий</b>

В целом мне все понравилось: твоя работа выполнена на хорошем уровне с минимальными помарками.
Также работа выглядит аккуратной и хорошо оформленной.

К работе нет серьезных замечаний, и проект может быть принят. Возвращаю тебе работу только для знакомства с моими комментариями. Возможно, ты решишь что-то улучшить или у тебя появятся вопросы.
     
Жду работу на повторное ревью.
</div>

<div class="alert alert-info", style="color:brown">

**Еще раз спасибо за ревью и замечания. Проекты с исправленными замечаниями становятся лучшими шпаргалками на будущее)**
    
</div>

<img src="http://s3.amazonaws.com/pix.iemoji.com/images/emoji/apple/ios-12/256/waving-hand.png" align=left width=44, heigth=44>
<div class="alert alert-info">
<b> Заключительный комментарий v2</b>

Привет.
Вижу, что ты оперативно отреагировал на мои комментарии. 
И большое спасибо за обратную связь )).
    
Проект принят, поздравляю! И желаю дальнейших успехов!
</div>